<h1>Creating a New Thorn, Part 2</h1>
So far we've created an empty thorn which does nothing. Never before did doing nothing feel like such an accomplishment!

We can use it as a template for something more useful. To that end, we're going to create a thorn which computes an "energy," the sum of the squares of the wave velocities at each point on the grid. First, we create the source file itself. It's simple enough.

In [ ]:
%cd ~/CactusFW2

In [ ]:
%%writefile ./arrangements/FunwaveUtils/EnergyCalc/src/energy.cc
// We pretty much always want to include these 3 headers
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>

void compute_energy(CCTK_ARGUMENTS) // Cactus functions always have this prototype
{
    DECLARE_CCTK_ARGUMENTS; // Declare all grid functions (from interface.ccl)
    DECLARE_CCTK_PARAMETERS; // Declare all parameters (from param.ccl)
    
    // Note that even though this is really a 2-d calculation, Cactus
    // thinks of it as 3-d with 1 zone in the z direction.
    for(int k=0;k<cctk_lsh[2];k++) { // loop over the z direction
        for(int j=0;j<cctk_lsh[1];j++) { // loop over the y direction
            for(int i=0;i<cctk_lsh[0];i++) { // loop over the x direction
                int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
                energy[cc] = u[cc]*u[cc]+v[cc]*v[cc];
            }
        }
    }
}

Next we update the makefile so that it will build. Are we done? Let's try to compile.

In [ ]:
%%writefile arrangements/FunwaveUtils/EnergyCalc/src/make.code.defn
# Main make.code.defn file for thorn Energy

# Source files in this directory
SRCS = energy.cc

# Subdirectories containing source files
SUBDIRS =


In [ ]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./my_thorns.th | cat -

Why did it fail? Two reasons. First, the grid function "energy" was not defined. Second, the code did not know how to access the velocity fields, "u" and "v."

To overcome the first problem, we decalare the "energy_group" below. To fix the second, we simply inherit from thorn Funwave where the velocities "u" and "v" are defined.

In [ ]:
%%writefile ./arrangements/FunwaveUtils/EnergyCalc/interface.ccl
## Interface definitions for thorn Energy
inherits: Funwave
## An implementation name is required for all thorns. No
## two thorns in a configuration can implement the same
## interface.
implements: EnergyCalc

## the groups declared below can be public, private, or protected.
public:

## A group defines a set of variables that are allocated together
## and share common properties, i.e. timelevels, tags such as the
## Prolongation=None tag. The type tag can take on the values
## GF, Scalar, or Array.

## Note that the number of timelevels can be an integer parameter
## GF stands for "Grid Function" and refers to a distributed array
## data structure.
#cctk_real force_group type=GF timelevels=3 tags='Prolongation="None"'
#{
#  force1, force2
#}

cctk_real energy_group type=GF
{
    energy
}

## Scalars are single variables that are available on all processors.
#{
#  scalar1, scalar2
#}


With this change, everything should compile...

In [ ]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./my_thorns.th | cat -

So far so good. We've created a new thorn, including all the boiler plate code. We also filled in some source code and succeeded in compiling it. That's great. The only problem is, our thorn won't actually run. It's not scheduled to do so.

In [ ]:
%%writefile ./arrangements/FunwaveUtils/EnergyCalc/schedule.ccl
## Schedule definitions for thorn Energy

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## These commented examples correspond to the commented
## examples in the interface file above.
# storage: force_group[3]
# storage: scalar_group

## Schedule a function defined in this thorn to run at the beginning
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).
#SCHEDULE init_function at CCTK_INIT
#{
#  LANG: C
#}"Do some initial stuff"

schedule compute_energy in CCTK_ANALYSIS
{
    LANG: C
} "Compute the Energy for Analysis" # This comment is required!

What does CCTK_ANALYSIS mean? Cactus schedules it's work in several bins. While there are many of them, here we only consider three:

1) CCTK_INITIAL - This runs once at the beginning. Initialize your grid functions here

2) CCTK_EVOL - Evolve a single timestep forward. This step will run repeatedly until the simulation finishes. Actually, this is rarely used directly. Usually, MoL_CalcRHS is used instead. More on that in a little bit.

3) CCTK_ANALYSIS - Called right before output routines. This ensures values are filled in sensibly. Call it to calculate values that you want to look at, but which aren't necessarily needed for the evolution.

In [ ]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./my_thorns.th | cat -

Now, let's find out what our code does! First, we'll need to create a par file. It will be similar to the one we created
in the first lesson. We will, however, make two small modifications.

In [ ]:
%%writefile ~/CactusFW2/wave2.par

#Reorder the parameters for easy comparison to the input.txt in example 3
ActiveThorns = "
  CoordBase FunWave FunwaveCoord CartGrid3D Carpet CarpetIOASCII
  CartGrid3D IOUtil CarpetIOBasic CarpetSlab Boundary SymBase MoL
  CarpetReduce LocalReduce InitBase CarpetLib LoopControl Tridiagonal
  CarpetIOScalar EnergyCalc "

#----------------------------------------------------
# Flesh and CCTK parameters
#----------------------------------------------------

# flesh
Cactus::cctk_run_title = "Test Run"
Cactus::cctk_show_schedule = "yes"
Cactus::cctk_itlast = 300
Cactus::allow_mixeddim_gfs = "yes"

# CartGrid3D
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

CoordBase::boundary_size_x_lower     = 3
CoordBase::boundary_size_x_upper     = 3
CoordBase::boundary_size_y_lower     = 3
CoordBase::boundary_size_y_upper     = 3
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_x_lower = 1
CoordBase::boundary_shiftout_x_upper = 1
CoordBase::boundary_shiftout_y_lower = 1
CoordBase::boundary_shiftout_y_upper = 1
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# Carpet
Carpet::domain_from_coordbase = "yes"
Carpet::ghost_size_x = 3
Carpet::ghost_size_y = 3
Carpet::ghost_size_z = 1
carpet::adaptive_stepsize = yes

# MoL
MoL::ODE_Method = "RK3"
MoL::disable_prolongation        = "yes"

# the output dir will be named after the parameter file name
IO::out_dir = $parfile
IO::out_fileinfo="none"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "FunWave::eta FunWave::u FunWave::v"

#IOASCII::out1D_every = 1
#IOASCII::out1d_vars = "FunWave::eta Funwave::depth"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 30
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "FunWave::eta FunWave::u FunWave::v EnergyCalc::energy"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

IOScalar::outScalar_every = 1
IOScalar::outScalar_vars = "FunWave::eta FunWave::u FunWave::v"

#& = "Funwave::eta"

#----------------------------------------------------
# Funwave parameters
#----------------------------------------------------

# Funwave depth 
FunWave::depth_file_offset_x = 3
FunWave::depth_file_offset_y = 3
FunWave::depth_type = "flat"
FunWave::depth_format = "ele"
FunWave::depth_file = "/tmp/__depth__.txt"
FunWave::depth_flat = 0.8
#Funwave::test_depth_shore_x = 80
#Funwave::test_depth_island_x = 40
#Funwave::test_depth_island_y = 40
FunWave::depth_xslp = 10.0
FunWave::depth_slope = 0.05
FunWave::dt_size = 0
Funwave::generate_test_depth_data = true
Funwave::num_wave_components = 1
Funwave::wave_component_file = "/home/sbrandt/workspace/shi_funwave/example_2/fft/wavemk_per_amp_pha.txt"
Funwave::peak_period = 1

# import
Funwave::time_ramp = 1.0
Funwave::delta_wk = 0.5
Funwave::dep_wk = 0.45
Funwave::xc_wk = 3.0
Funwave::ywidth_wk = 10000.0
Funwave::tperiod = 1.0
Funwave::amp_wk = 0.0232
Funwave::theta_wk = 0.0
Funwave::freqpeak = 0.2
Funwave::freqmin = 0.1
Funwave::freqmax = 0.4
Funwave::hmo = 1.0
Funwave::gammatma = 5.0
Funwave::thetapeak = 10.0
Funwave::sigma_theta = 15.0

# Funwave wind forcing
Funwave::wind_force = false
Funwave::use_wind_mask = false
Funwave::num_time_wind_data = 2
Funwave::timewind[0] = 0
Funwave::wu[0] = 25
Funwave::wv[0] = 50
Funwave::timewind[1] = 1000
Funwave::wu[1] = 100
Funwave::wv[1] = 100
Funwave::boundary = funwave

# Funwave wave maker
FunWave::wavemaker_type = "ini_gau"
FunWave::xc = 26.5
FunWave::yc = 26.9
FunWave::amp =  2.0
FunWave::wid =  1
Funwave::wdep = 0.78
Funwave::xwavemaker = 25.0

# Funwave sponge 
FunWave::sponge_on = false
FunWave::sponge_west_width = 2.0
FunWave::sponge_east_width = 2.0
FunWave::sponge_north_width = 0.0
FunWave::sponge_south_width = 0.0
FunWave::sponge_decay_rate = 0.9
FunWave::sponge_damping_magnitude = 5.0

# Funwave dispersion (example 3 enables dispersion)
FunWave::dispersion_on = "true"
FunWave::gamma1 = 1.0
FunWave::gamma2 = 1.0
FunWave::gamma3 = 1.0
FunWave::beta_ref = -0.531
FunWave::swe_eta_dep = 0.80
FunWave::cd = 0.0

# Funwave numerics (MoL parameter controls time integration scheme)
FunWave::reconstruction_scheme = "fourth"
FunWave::riemann_solver = "HLLC"
FunWave::dtfac = 0.5
FunWave::froudecap = 10.0
FunWave::mindepth = 0.001
FunWave::mindepthfrc = 0.001
FunWave::enable_masks = "true"
Funwave::estimate_dt_on = "true"

FunwaveCoord::spherical_coordinates = false

ActiveThorns = "CarpetIOHDF5"
IOHDF5::out2D_xyplane_z = 0 
IOHDF5::out2D_every = 10
IOHDF5::out2D_vars = " 
  FunWave::eta
  FunWave::u
  FunWave::v
  Grid::Coordinates{out_every=1000000000}
"
IOHDF5::out2D_xz = no
IOHDF5::out2D_yz = no

What, exactly, did we change? We can use diff to find out.

In [ ]:
!diff wave.par wave2.par

In [ ]:
!rm -fr ~/simulations/wave2

In [ ]:
%cd ~/CactusFW2
!./simfactory/bin/sim create-run --procs 2 --num-threads 1 wave2.par

Where did things go wrong? We can re-run the simulation with gdb and get a stack trace to see.

In [ ]:
!gdb ./exe/cactus_sim --eval-command="run wave2.par" --eval-command=where < /dev/null

The segfault occurs when we first attempt to write the energy variable. The problem is, Cactus has not allocated storage
for the variable. To do that, we must edit the schedule file and add a storage declaration for the energy_group.

In [ ]:
%%writefile arrangements/FunwaveUtils/EnergyCalc/schedule.ccl
## Schedule definitions for thorn Energy

storage: energycalc::energy_group

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## These commented examples correspond to the commented
## examples in the interface file above.
# storage: force_group[3]
# storage: scalar_group

## Schedule a function defined in this thorn to run at the beginning
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).
#SCHEDULE init_function at CCTK_INIT
#{
#  LANG: C
#}"Do some initial stuff"

schedule compute_energy in CCTK_ANALYSIS
{
    LANG: C
} "Compute the Energy for Analysis" # This comment is required!

In [ ]:
!time ./simfactory/bin/sim build -j 2 --thornlist=./my_thorns.th

In [ ]:
!rm -fr ~/simulations/wave2

In [ ]:
!./simfactory/bin/sim create-run --procs 2 --num-threads 1 wave2.par

Now let's plot our energy function. Basically, we're just copying cells from the Cactus-Funwave notebook.

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.cm as cm
# https://matplotlib.org/examples/color/colormaps_reference.html
cmap = cm.gist_rainbow

In [ ]:
import os
dir = os.environ["HOME"]+"/simulations/wave2/output-0000/wave2/"
file_data = np.genfromtxt(dir+"energy.xy.asc")

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,12]),np.max(file_data[:,12])
for which in sets: 
    print("which=",which)
    g = file_data[file_data[:,0]==which,:]
    x = g[:,5]
    y = g[:,6]
    z = g[:,12]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

Exercise:
Make a copy of both "CreatingANewThorn" notebooks and edit them to create another new thorn, "LaplacianEta" which computes the Laplacian of the eta variable, which is a measure of the displacement of the surface of the water from rest. Like the "Energy" thorn, this quantity will be computed at analysis time.

The second derivative of a quantity is approximated by the finite difference formula:

$f''[x] = (f[x+dx] - 2 f[x] + f[x-dx])/dx^2 \approx ((f[x]+dx f'[x]+\frac{1}{2}dx^2 f[x]+...) - 2 f[x] + (f[x]-dx f'[x]+\frac{1}{2}dx^2 f[x]+...))/dx^2$

In terms of Cactus, f[x] and f[x+dx] might look something like this...
<pre>
  int cc = CCTK_GFINDEX3D(cctkGH,i,j,k)
  int cp1 = CCTK_GFINDEX3D(cctkGH,i+1,j,k)
  CCTK_REAL fx = f[cc]; // If this is f[x]
  CCTK_REAL fx1 = f[cp1]; // this is f[x+dx]
</pre>

Cactus provides an additional array of integers, like cctk_lsh, called cctk_delta_space, which provides the quantities dx, dy, and dz (these are cctk_delta_space[0], cctk_delta_space[1] and cctk_delta_space[2], respecitively).

Because this is a 2-d code, the Laplacian is

$\Delta^2 \eta = \left( \frac{d^2}{dx^2} + \frac{d^2}{dy^2} \right) \eta$

Note that you will not be able to calculate the value of the Laplacian at the borders of the grid, as that would result in a segfault. Please write zeroes in the borders instead.

Note also that Funwave defines grid variables dx and dy. You can use dx[cc] (where cc = CCTK_GFINDEX3D(cctkGH,i,j,k)) in place of cctk_delta_space[0] if you want to. You can't, however, redefined dx or dy.

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> and <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"> CCF 1539567</a></td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>